# Laboratorio 6.a

Classificazione basic/advanced. Si richiedere l’uso (o meno) del dataset su basicness per fare classificazione automatica (binaria, basic/advanced) su nuovi termini e/o synset presi in esame.

In [74]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [76]:
def open_json(file_name):
    with open(file_name, 'r') as file:
        data = json.load(file)

    df = pd.DataFrame(data['dataset'], columns=['dataset'])

    df[['synset', 'word', 'gloss']] = df['dataset'].str.split(':', expand=True)
    df = df.drop(columns=['dataset'])
    df = df.drop(columns=['synset'])
    df = df.drop(columns=['gloss'])
    df['word'] = df['word'].str.replace(' \| Definition', '', regex=True)
    df['word_embedding'] = None
    df['target'] = data['answers']

    new_rows = []  # Creazione di una lista per le nuove righe
    # per ogni df['word'] splitta tramite "," e crea due righe con la stessa etichetta
    for i in range(len(df)):
        if ',' in df['word'][i]:
            new_row = {'word': df['word'][i].split(', ')[1], 'target': df['target'][i]}
            new_rows.append(new_row)
            df['word'][i] = df['word'][i].split(', ')[0]
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)  # aggiunta delle nuove righe al df
    return df


In [77]:
file_name = "basic_advanced/1.json"
df = open_json(file_name)
df.head()

,word,word_embedding,target
0,war,None,basic
1,fiefdom,None,advanced
2,bed,None,basic
3,return on invested capital,None,advanced
4,texture,None,basic


## Generazione degli embeddings

In [78]:
def load_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.strip().split()
            word = values[0]
            vector = [float(val) for val in values[1:]]
            embeddings[word] = vector
    return embeddings

glove_path = 'glove.6B/glove.6B.50d.txt'
embeddings = load_embeddings(glove_path)

In [79]:
for i in range(len(df)):
    try:
        df['word_embedding'][i] = embeddings[df['word'][i]]
    except:
        df = df.drop(i)

df.head()

,word,word_embedding,target
0,war,"[0.36544, -0.15746, -0.23966, -1.0307, -0.0706...",basic
1,fiefdom,"[0.66869, -0.3902, -0.27852, 0.11272, -0.3041,...",advanced
2,bed,"[0.75344, 0.96383, 0.10188, -0.67978, 0.38461,...",basic
4,texture,"[0.046497, -0.19038, -1.2709, 0.36144, 1.0203,...",basic
5,news,"[-0.20825, 0.47786, 0.52196, 1.0587, -0.10045,...",basic


## Training/Test set

In [92]:
X_train, X_test, y_train, y_test = train_test_split(df['word_embedding'], df['target'], test_size=0.2, random_state=42)

X_train = np.array([embedding for embedding in X_train])
X_test = np.array([embedding for embedding in X_test])

## Regressione logistica

In [96]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [97]:
y_pred = model.predict(X_test)
accuracy = model.score(X_test, y_test)

In [98]:
print(accuracy)

0.8099173553719008
